In [1]:
# https://datos.gob.ar/dataset/otros-resultados-provisorios-elecciones-generales-2019/archivo/otros_12.1

import pandas as pd

In [5]:
year = 2019
election = "first_round"

df = pd.read_csv(f"../data/Argentina/{year}/mesas_agrp_politicas.dsv", delimiter="|")
df

C:\Users\cnava\AppData\Local\Temp\ipykernel_38824\876347767.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/Argentina/2019/mesas_agrp_politicas.dsv", delimiter="|")


,CODIGO_DISTRITO,CODIGO_SECCION,CODIGO_CIRCUITO,CODIGO_MESA,CODIGO_CATEGORIA,CODIGO_AGRUPACION,CODIGO_LISTA,VOTOS_AGRUPACION
0,1,1001,1001000001,0100100001X,100000000000,131,000100000000000.131.A.000100000000000.131,3
1,1,1001,1001000001,0100100001X,100000000000,133,000100000000000.133.A.000100000000000.133,14
2,1,1001,1001000001,0100100001X,100000000000,135,000100000000000.135.A.000100000000000.135,115
3,1,1001,1001000001,0100100001X,100000000000,136,000100000000000.136.A.000100000000000.136,114
4,1,1001,1001000001,0100100001X,100000000000,137,000100000000000.137.A.000100000000000.137,14
...,...,...,...,...,...,...,...,...
1772804,24,24005,24005000021,2400500484X,324000000000,24-203,000324000000000.24-203.A.000324000000000.24-203,2
1772805,24,24005,24005000021,2400500484X,324000000000,24-501,000324000000000.24-501.A.000324000000000.24-501,44
1772806,24,24005,24005000021,2400500484X,324000000000,24-502,000324000000000.24-502.A.000324000000000.24-502,15
1772807,24,24005,24005000021,2400500484X,324000000000,24-503,000324000000000.24-503.A.000324000000000.24-503,14


In [15]:
df_candidates = pd.read_csv("../data/Argentina/2019/descripcion_postulaciones.dsv", delimiter="|")
df_candidates = df_candidates[df_candidates["NOMBRE_CATEGORIA"] == "Presidente y Vicepresidente de la República"]
df_candidates = df_candidates[["CODIGO_CATEGORIA", "CODIGO_AGRUPACION", "NOMBRE_LISTA"]]
df_candidates

,CODIGO_CATEGORIA,CODIGO_AGRUPACION,NOMBRE_LISTA
503,100000000000,135,Mauricio Macri - Miguel Pichetto
504,100000000000,131,Juan Gómez - Cynthia Hotton
505,100000000000,136,Alberto Fernández - Cristina Fernández
506,100000000000,133,Nicolás Del Caño - Romina Del Plá
507,100000000000,137,Roberto Lavagna - Juan Urtubey
508,100000000000,87,José Espert - Luis Rosales


In [17]:
df = pd.merge(df, df_candidates, on=["CODIGO_CATEGORIA", "CODIGO_AGRUPACION"])
df = df.rename(columns={
    "VOTOS_AGRUPACION": "value",
    "NOMBRE_LISTA": "candidate",
    "CODIGO_MESA": "polling_id"
})

In [18]:
tt = df.groupby(["polling_id", "candidate"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "rate", "value"]]

tt["rank"] = tt.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

In [19]:
df_filtered = tt[["polling_id", "candidate", "value", "rank", "rate"]].copy()
election = "first_round"

df_filtered.to_csv(f"../data_output/Argentina/{year}_{election}.csv.gz", compression="gzip", index=False)

In [23]:
df_location = df[["polling_id", "CODIGO_DISTRITO", "CODIGO_SECCION", "CODIGO_CIRCUITO"]].drop_duplicates()
df_location = df_location.rename(columns={
    "CODIGO_DISTRITO": "district_id",
    "CODIGO_SECCION": "section_id",
    "CODIGO_CIRCUITO": "circuit_id"
})

df_location.to_csv(f"../data_output/Argentina/{year}_{election}_location.csv.gz", compression="gzip", index=False)